# Scenario 1
18 month hackathon (02/26/2024)

### Load dependencies and interfaces

In [1]:
import os
import pyciemss
import torch
import pandas as pd
smoke_test = ('CI' in os.environ)

### Select models and data

In [2]:
MODEL_PATH = "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration/main/data/models/"
DATA_PATH = r"https://raw.githubusercontent.com/ciemss/program-milestones/epi-scenario-1/18-month-milestone/hackathon/epi/Scenario%201%20Supplemental/"

model1 = os.path.join(MODEL_PATH, "SEIRD_base_model01_petrinet.json")
dataset1 = os.path.join(DATA_PATH, "UK_compartments.csv")

In [3]:
# # Load datasets 
# df = pd.read_csv(dataset1, index_col=0)

# # Group by 't' and 'compartment', then sum 'y' values
# grouped = df.groupby(['t', 'compartment'])['y'].sum()

# # Reshape the DataFrame and fill missing values with 0
# df_new = grouped.unstack(fill_value=0)

# # Make a column 'I' which sums 'ICase' and 'IMild' and delete the original columns
# df_new['I'] = df_new['ICase'] + df_new['IMild']
# df_new.drop(columns=['ICase', 'IMild'], inplace=True)

# # Output to CSV
# df_new.to_csv('UK_SEIRD.csv')

# # Redefine the dataset
# dataset1 = 'UK_SEIRD.csv'

### Set parameters for sampling

In [4]:
start_time = 0.0
end_time = 200.
logging_step_size = 10.0
num_samples = 10

## Sample interface
Take `num_samples` number of samples from the (prior) distribution invoked by the chosen model.

### Sample from model 1

In [5]:
result1 = pyciemss.sample(model1, end_time, logging_step_size, num_samples, start_time=start_time, solver_method='euler')
result1["unprocessed_result"]

{'persistent_beta': tensor([0.2931, 0.7923, 0.2567, 0.3437, 0.3974, 0.5351, 0.6302, 0.2546, 0.4345,
         0.1463]),
 'persistent_death': tensor([0.0065, 0.0036, 0.0088, 0.0092, 0.0037, 0.0047, 0.0092, 0.0057, 0.0049,
         0.0028]),
 'persistent_gamma': tensor([0.1001, 0.3520, 0.3449, 0.3369, 0.3109, 0.1558, 0.1856, 0.3753, 0.4033,
         0.1451]),
 'persistent_I0': tensor([ 2.8867,  9.9100, 11.6928,  9.4339, 11.5022,  1.8330,  8.4244,  4.0940,
         12.4576,  1.3564]),
 'D_state': tensor([[ 1.8740e-02,  6.6791e-01, -1.6908e-01,  5.8429e+00, -9.3116e+00,
           5.7476e+01, -1.5377e+02,  6.5266e+02, -2.1031e+03],
         [ 1.2435e-01,  1.0657e+00, -7.2607e-01,  2.1564e+01, -9.6765e+01,
           7.3620e+02, -4.5074e+03,  2.9975e+04, -1.9019e+05],
         [ 3.5380e-01,  2.5133e+00, -5.0425e+00,  3.0720e+01, -1.3123e+02,
           6.0643e+02, -2.7507e+03,  1.2533e+04, -5.6975e+04],
         [ 2.9328e-01,  2.7071e+00, -5.1555e+00,  3.7430e+01, -1.6697e+02,
           8.3

In [6]:
result1['data'].head()

,timepoint_id,sample_id,persistent_beta_param,persistent_death_param,persistent_gamma_param,persistent_I0_param,D_state,E_state,I_state,R_state,S_state,infected_observable_state,dead_observable_state
0,0,0,0.293087,0.006487,0.100081,2.886689,0.018740,-51.539497,99.997673,2.870276,19339990.0,99.997673,0.018740
1,1,0,0.293087,0.006487,0.100081,2.886689,0.667913,370.389099,-128.929352,102.299377,19339696.0,-128.929352,0.667913
2,2,0,0.293087,0.006487,0.100081,2.886689,-0.169081,-933.452698,926.076660,-25.896912,19340074.0,926.076660,-0.169081
3,3,0,0.293087,0.006487,0.100081,2.886689,5.842897,4114.398438,-2334.378418,894.914307,19337360.0,-2334.378418,5.842897
4,4,0,0.293087,0.006487,0.100081,2.886689,-9.311605,-13012.419922,10287.876953,-1426.191162,19344200.0,10287.876953,-9.311605


## Calibrate interface
Calibrate a model to a dataset by mapping model state varibale or observables to columns in the dataset

In [7]:
### Choose data to calibrate on
# Load CSV 'UK_SEIRD.csv'
df = pd.read_csv(dataset1, index_col=0)

# Output until t = 40 to CSV
df[df['t'] <= 40].to_csv('UK_SEIRD_40.csv')

,D,E,R,S,I
t,,,,,
0.2,0,200,0,678859840,0
0.4,0,200,0,678859840,0
0.6,0,198,0,678859840,2
0.8,0,198,1,678859840,1
1.0,0,194,1,678859840,5


In [ ]:
data_mapping = {"I": "I", "D": "D"} # data_mapping = "column_name": "observable/state_variable"
num_iterations = 10 
calibrated_results = pyciemss.calibrate(model1, dataset1, data_mapping=data_mapping, num_iterations=num_iterations)
parameter_estimates = calibrated_results["inferred_parameters"]
calibrated_results

{'inferred_parameters': AutoGuideList(
   (0): AutoDelta()
   (1): AutoLowRankMultivariateNormal()
 ),
 'loss': 248.0983643233776}

In [ ]:
parameter_estimates()

{'persistent_beta_c': tensor(0.4845, grad_fn=<ExpandBackward0>),
 'persistent_kappa': tensor(0.3275, grad_fn=<ExpandBackward0>),
 'persistent_gamma': tensor(0.3196, grad_fn=<ExpandBackward0>),
 'persistent_hosp': tensor(0.0939, grad_fn=<ExpandBackward0>),
 'persistent_death_hosp': tensor(0.0549, grad_fn=<ExpandBackward0>),
 'persistent_I0': tensor(5.9766, grad_fn=<ExpandBackward0>)}